# 1. Set Environments

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="Machine Translator_01"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

In [2]:
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 2. Set Datasets

In [3]:
from datasets import Dataset, DatasetDict
import pickle
import os
import time


with open('./data/train_data_300.pkl', 'rb') as file:
    train_data = pickle.load(file)
len(train_data)

with open('./data/validation_data_28.pkl', 'rb') as file:
    test_data = pickle.load(file)
len(test_data)

train_dataset = Dataset.from_list(train_data)
test_dataset = Dataset.from_list(test_data)

# DatasetDict로 "train"과 "test" 데이터셋 묶기
dataset_dict = DatasetDict({
        'train': train_dataset,
        'test': test_dataset
    })

# 3. Checkpoints Listing

In [4]:
checkpoint_dir = "./checkpoint/translate_machine_llama3ko_intsuct_origindata300_02"

# 디렉터리 이름에서 숫자를 추출하는 함수
def extract_number(directory_name):
    return int(directory_name.split('-')[-1])


# 디렉터리 항목을 숫자 순서대로 정렬
checkpoints = []
with os.scandir(checkpoint_dir) as entries:
    for entry in entries:
        if entry.is_dir():
            checkpoints.append(entry.name)

# 숫자 순서대로 정렬
checkpoints.sort(key=extract_number)

# 4. Evaluate

### (1) Function

- 각 metric 연산을 위해 패키지 설치 필요
```
$ pip install sacrebleu
$ pip install unbabel-comet
$ pip install bert-score
```
- comet과 bert_score는 gpu연산도 가능하지만 차후 비동기 형태로 gpu는 생성을 계속하고, cpu는 metric연산을 할수 있도록 하기 위해 metric은 cpu연산을 하도록 인수설정

In [5]:
import evaluate
import re
import pandas as pd
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import torch

bleu_metric = evaluate.load("sacrebleu")
comet_metric = evaluate.load("comet")
bert_metric = evaluate.load("bertscore")

model_id = "beomi/Llama-3-KoEn-8B-Instruct-preview"
device_map = {"": 0}
cache_model_dir="/mnt/t7/.cache/huggingface/models"

def make_prompt(text):
    return f'''Translate input sentence to Korean
### Input: {text}
### Translated:'''


def clean_text(text):
    text = re.sub(r'\(.*?\)', '', text)
    return text.strip()

def count_mask(batchs):
    mask = []
    for batch in batchs["attention_mask"]:
        count = torch.sum(batch == 0).item()
        mask.append(count)

    return mask

def counter_terms(terms, text):
    if not isinstance(terms, list):
        terms = terms.split(", ")

    return sum(text.lower().count(term.lower()) for term in terms)

def generate(inputs, tokenizer, model):
    examples = []

    for input in inputs:
        prompt = make_prompt(input)
        examples.append(prompt)

    example_batch = tokenizer(examples, return_tensors="pt", padding=True).to(model.device)

    mask = count_mask(example_batch)

    with torch.cuda.amp.autocast():
        output_tokens = model.generate(**example_batch, max_new_tokens = 512, pad_token_id=tokenizer.pad_token_id)

    outputs = [tokenizer.decode(t[len(tokenizer.encode(examples[i])) + mask[i]:], skip_special_tokens=True) for i, t in enumerate(output_tokens)]

    return outputs


def evaluate_score(datasets, outputs):
    if not len(datasets["korean"]) == len(outputs):
        pass

    inputs = datasets["english"]
    labels = datasets["korean"]
    terms = datasets["terms"]

    results = []

    for input, output, label, term in zip(inputs, outputs, labels, terms):
        
        label_clean = clean_text(label)
        prediction_clean = clean_text(output)
        bleu_result = bleu_metric.compute(predictions=[prediction_clean], references=[label_clean])
        comet_result = comet_metric.compute(predictions=[prediction_clean], references=[label_clean], sources=[input], gpus = 0)
        bert_result = bert_metric.compute(predictions=[prediction_clean], references=[label_clean], lang="ko", device = "cpu")
        print(f"Label: {label_clean}")
        print(f"Output: {prediction_clean}")
        print("*" * 100)

        input_count = counter_terms(term, input)
        predic_count = counter_terms(term, output)

        weight = 1.0 if predic_count > input_count else predic_count / input_count

        results.append((weight, bleu_result['score'], comet_result['scores'][0], bert_result['f1'][0]))
    
    return results


model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=BitsAndBytesConfig(load_in_8bit=True), device_map='auto', cache_dir=cache_model_dir)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Lightning automatically upgraded your loaded checkpoint from v1.8.3.post1 to v2.3.3. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../../../../home/aeolian83/.cache/huggingface/hub/models--Unbabel--wmt22-comet-da/snapshots/371e9839ca4e213dde891b066cf3080f75ec7e72/checkpoints/model.ckpt`
Encoder model frozen.
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

### (2) Evaluate Model

In [6]:
# 시작 시간 기록
start_time = time.time()

metric_scores = {}

for checkpoint in checkpoints[10:20]:
    print('#' * 50)
    print(checkpoint)
    print('#' * 50)

    peft_model_id = os.path.join(checkpoint_dir, checkpoint)
    tokenizer = AutoTokenizer.from_pretrained(peft_model_id)
    tokenizer.padding_side = "left"
    model.resize_token_embeddings(len(tokenizer))

    loaded_model = PeftModel.from_pretrained(model=model, model_id=peft_model_id)

    results = []
    for i in range(0, len(dataset_dict["test"]), 6):
        dataset = dataset_dict["test"][i:i+6]

        outputs = generate(dataset["english"], tokenizer, loaded_model)

        results += evaluate_score(dataset, outputs)
        print(results)

    print(results)

    weights, bleus, comet_score, bert_f1 = zip(*results)
    score = {
        "weight": np.mean(weights),
        "bleu": np.mean(bleus),
        "comet": np.mean(comet_score),
        "bert_score_f1": np.mean(bert_f1),
        }
    metric_scores[checkpoint] = score

    del peft_model_id, tokenizer, loaded_model
    torch.cuda.empty_cache()

df = pd.DataFrame.from_dict(metric_scores, orient='index')

# 종료 시간 기록
end_time = time.time()

print(f"Execution Time: {end_time - start_time} seconds")
print(df)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


##################################################
checkpoint-110
##################################################


GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Label: 그룹 희소성은 다중선형 대수에서 변수 그룹 내의 희소성 패턴을 촉진하는 개념입니다. 이 기술은 특히 고차원 데이터와 관련된 응용 분야에서 유용하며, 관련된 특징 그룹을 식별하는 데 도움을 줍니다. 팩터 그래프에서 그룹 희소성은 그래프 구조의 복잡성을 줄여 추론 알고리즘의 효율성을 높일 수 있습니다. 다중선형 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기초를 제공합니다. 그룹 희소성과 다중선형 대수를 활용함으로써 팩터 그래프는 대규모 문제를 더 효과적으로 처리할 수 있습니다.
Output: 그룹 희소성은 변수 집합 내에서 희소 패턴을 촉진하는 다변수 대수의 개념입니다. 이 기술은 고차원 데이터를 다루는 응용 프로그램에서 특히 유용하며, 관련된 특징 집합을 식별하는 데 도움을 줍니다. 인자 그래프에서 그룹 희소성은 그래프 구조의 복잡성을 줄임으로써 추론 알고리즘의 효율성을 높일 수 있습니다. 다변수 대수는 이러한 그룹 간의 상호작용을 이해하고 조작하는 수학적 기반을 제공합니다. 그룹 희소성과 다변수 대수를 활용하여 인자 그래프는 대규모 문제를 더 효과적으로 처리할 수 있습니다.
****************************************************************************************************


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Label: 히든 마코프 모델은 시퀀스 분석과 음성 인식에서 시계열 데이터를 모델링하는 능력 때문에 널리 사용됩니다. 몬테카를로 방법은 HMMs와 함께 복잡한 확률 계산과 시뮬레이션을 수행하는 데 자주 사용됩니다. 유전 알고리즘은 진화 전략을 활용하여 HMMs의 매개변수를 최적화하는 또 다른 접근법을 제공합니다. 몬테카를로 방법과 유전 알고리즘을 통합함으로써 연구자들은 HMMs의 성능과 정확성을 향상시킬 수 있습니다. 이러한 조합은 견고한 통계 모델과 효율적인 최적화 기술이 필요한 분야에서 특히 유용한 것으로 입증되었습니다.
Output: 히든 마코프 모델은 시간 시리즈 데이터를 모델링할 수 있는 능력 때문에 시퀀스 분석 및 음성 인식에서 널리 사용됩니다. 몬테카를로 방법은 히든 마코프 모델과 함께 복잡한 확률 계산 및 시뮬레이션을 수행하는 데 자주 사용됩니다. 유전 알고리즘은 진화 전략을 활용하여 히든 마코프 모델의 매개변수를 최적화하는 또 다른 접근 방식을 제공합니다. 몬테카를로 방법과 유전 알고리즘을 통합함으로써 연구자들은 히든 마코프 모델의 성능과 정확성을 향상시킬 수 있습니다. 이 조합은 견고한 통계 모델과 효율적인 최적화 기법이 필요한 분야에서 특히 유용함이 입증되었습니다.
****************************************************************************************************


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Label: 마르코프 랜덤 필드는 컴퓨터 비전과 공간 통계에서 서로 다른 변수 간의 의존성을 나타내는 데 사용되는 강력한 모델입니다. 조건부 랜덤 필드는 관찰된 데이터를 통합하여 MRFs를 확장하며, 시퀀스 라벨링 작업에 특히 유용합니다. 마르코프 랜덤 필드와 조건부 랜덤 필드는 모두 쌍별 마르코프 네트워크를 사용하여 표현할 수 있습니다. 쌍별 마르코프 네트워크는 복잡한 상호작용을 쌍별 관계로 분해하여 이들 분야에서의 계산을 단순화합니다. 이 접근법은 마르코프 랜덤 필드와 조건부 랜덤 필드 모두에서 더 효율적인 추론을 가능하게 합니다.
Output: 마코프 랜덤 필드는 컴퓨터 비전과 공간 통계에서 다양한 변수 간의 종속성을 표현하는 강력한 모델입니다. 조건부 랜덤 필드는 관찰된 데이터를 통합하여 마코프 랜덤 필드를 확장하며, 특히 시퀀스 레이블링 작업에 유용합니다. 마코프 랜덤 필드와 조건부 랜덤 필드는 쌍별 마코프 네트워크를 사용하여 변수 쌍 간의 관계를 모델링할 수 있습니다. 쌍별 마코프 네트워크는 복잡한 상호작용을 쌍별 관계로 분해하여 이러한 필드에서의 계산을 단순화합니다. 이 접근법은 마코프 랜덤 필드와 조건부 랜덤 필드에서 더 효율적인 추론을 가능하게 합니다.
****************************************************************************************************


/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.


Label: 무한 은닉 마르코프 모델은 은닉 상태의 수를 무한대로 허용하여 전통적인 은닉 마르코프 모델을 확장합니다. 이는 스틱-브레이킹 프로세스를 사용하여 무한한 수의 구성 요소에 대한 분포를 구성하는 방법을 통해 달성됩니다. 스틱-브레이킹 프로세스는 iHMMs가 데이터에 따라 동적으로 모델의 복잡성을 조정할 수 있게 합니다. 순서 교환 가능 모델은 데이터 포인트의 순서가 결합 확률에 영향을 미치지 않는다고 가정하며, 순서가 중요한 역할을 하지 않는 시퀀스를 처리하기 위해 iHMMs와 함께 자주 사용됩니다. 무한 은닉 마르코프 모델과 순서 교환 가능 모델의 조합은 복잡한 데이터를 보다 유연하고 확장 가능하게 모델링할 수 있게 합니다.
Output: 무한 히든 마르코프 모델은 무한 개수의 히든 상태를 허용하여 전통적인 히든 마르코프 모델을 확장합니다. 이는 스틱 브레이킹 프로세스를 사용하여 무한 개의 구성 요소에 대한 분포를 구성하는 방법을 제공함으로써 달성됩니다. 스틱 브레이킹 프로세스는 iHMMs가 데이터에 기초하여 모델의 복잡성을 동적으로 적응할 수 있게 합니다. 교환 가능한 모델은 데이터 포인트의 순서가 결합 확률에 영향을 미치지 않는다고 가정하며, 이는 iHMMs와 함께 사용되어 순서가 중요하지 않은 시퀀스를 처리합니다. 무한 히든 마르코프 모델과 교환 가능한 모델의 결합은 복잡한 데이터를 더 유연하고 확장 가능한 방식으로 모델링할 수 있게 합니다.
****************************************************************************************************
[(1.0, 50.61673222493086, 0.9079117774963379, 0.9444258809089661), (1.0, 55.212019270637796, 0.9273369312286377, 0.952081561088562), (1.0, 51.56749901680131, 0.9160967469215393, 0.9297

In [7]:
print(f"Execution Time: {end_time - start_time} seconds")
print(df)

Execution Time: 2108.1940536499023 seconds
                  weight       bleu     comet  bert_score_f1
checkpoint-110  0.793141  48.177844  0.910979       0.925299
checkpoint-120  0.876219  47.962463  0.908386       0.928496
checkpoint-130  0.872251  48.172708  0.907940       0.928183
checkpoint-140  0.823087  50.928724  0.908978       0.937717
checkpoint-150  0.908603  48.674255  0.907708       0.931905
checkpoint-160  0.903784  49.431061  0.910814       0.934415
checkpoint-170  0.884283  49.087142  0.911340       0.932130
checkpoint-180  0.817616  49.667802  0.911561       0.933061
checkpoint-190  0.834821  50.013060  0.909828       0.932986
checkpoint-200  0.908801  48.209579  0.909444       0.932092
